In [14]:
# Installing necessary packages

%pip install pandas
%pip install selenium
%pip install webdriver-manager
%pip install spotipy 
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importing libraries and installing Chrome webdriver

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [29]:
#create chromeoptions instance
options = webdriver.ChromeOptions()


#provide location where chrome stores profiles
options.add_argument(r"--user-data-dir=C:/Users/ajaym/AppData/Local/Google/Chrome/User Data")

#provide the profile name with which we want to open browser
options.add_argument(r'--profile-directory=Profile 3')

## attempts to save file to specific folder
prefs = {"download.default_directory":"D:/Disseratation_data/"}
options.add_experimental_option('prefs', prefs)

#options.add_argument('--headless')

#options.add_argument(r'--download.default_directory=D:/Disseratation_data/')

In [387]:
# Automated code to open chrome browser, navigate to page and click on download button 

from datetime import date, timedelta
start_date = date(2022, 12, 31)
end_date = date(2023, 1, 1)
time_delta = timedelta(days=1)

driver = webdriver.Chrome(options=options)
while start_date <= end_date:
    driver.get(
        f'https://charts.spotify.com/charts/view/regional-gb-daily/{start_date}')
    # Click on Download Button
    time.sleep(2)
    driver.find_element(By.XPATH,"//button[@aria-labelledby='csv_download']").click()
    start_date += time_delta
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00EBA813+48355]
	(No symbol) [0x00E4C4B1]
	(No symbol) [0x00D55358]
	(No symbol) [0x00D3D293]
	(No symbol) [0x00D9E37B]
	(No symbol) [0x00DAC473]
	(No symbol) [0x00D9A536]
	(No symbol) [0x00D782DC]
	(No symbol) [0x00D793DD]
	GetHandleVerifier [0x0111AABD+2539405]
	GetHandleVerifier [0x0115A78F+2800735]
	GetHandleVerifier [0x0115456C+2775612]
	GetHandleVerifier [0x00F451E0+616112]
	(No symbol) [0x00E55F8C]
	(No symbol) [0x00E52328]
	(No symbol) [0x00E5240B]
	(No symbol) [0x00E44FF7]
	BaseThreadInitThunk [0x76BD7D59+25]
	RtlInitializeExceptionChain [0x77BAB79B+107]
	RtlClearBits [0x77BAB71F+191]


In [71]:
# Reading daily Top 200 songs csv files and combining into a single dataframe

start_date = date(2019, 1, 1)
end_date = date(2022, 12, 31)

daily_200_combined = pd.DataFrame()
while start_date <= end_date:
    temp_df = pd.read_csv(f'D:/Disseratation_data/regional-gb-daily-{start_date}.csv')
    temp_df['Date'] = start_date
    daily_200_combined = pd.concat([daily_200_combined, temp_df], axis=0)
    start_date += time_delta

In [72]:
# Verifying the length of the chart data to be of 4*365 + 1 day times 200

len(daily_200_combined)

292200

In [73]:
# Saving the combined chart data

daily_200_combined.to_csv('D:/Disseratation_data/combined_chart_data.csv', index = False)

In [5]:
daily_200_combined.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,days_on_chart,streams,Date
0,1,spotify:track:25sgk305KZfyuqVBQIahim,Ava Max,Sweet but Psycho,Atlantic Records,1,1,89,334456,2019-01-01
1,2,spotify:track:2rPE9A1vEgShuZxxzR2tZH,Ariana Grande,"thank u, next",Republic Records,1,2,59,317204,2019-01-01
2,3,spotify:track:3KkXRkHbMCARz0aVfEt68P,"Post Malone, Swae Lee",Sunflower - Spider-Man: Into the Spider-Verse,Universal Records,3,3,76,255033,2019-01-01
3,4,spotify:track:5p7ujcrUXASCNwRaWNHR1C,Halsey,Without Me,Capitol Records,2,4,89,232117,2019-01-01
4,5,spotify:track:6MWtB6iiXyIwun0YzU6DFP,Post Malone,Wow.,Republic Records,4,6,9,202739,2019-01-01


In [6]:
# Creating a list of unique uri's

song_uri_list = daily_200_combined['uri'].unique()

In [7]:
# Number of unique songs in UK Top 200 list during 2019 - 2022 

len(song_uri_list)

6473

In [21]:
# Authorising Spotify API usuage using client id and secret

client_credentials_manager = SpotifyClientCredentials(client_id='db8da9db1d5f40ceaed373b3bbfd94cc', client_secret='529377dbbff64685af3bb6e94578f80f')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [25]:
# Gathering audio features of songs through Spotify API

acoustic_properties_df = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
lower_limit = 0
upper_limit = len(song_uri_list)
for i in range(100, upper_limit, 100):
    af = sp.audio_features(song_uri_list[lower_limit:min(i, upper_limit)])
    for j in range(lower_limit, min(i, upper_limit)):
        acoustic_properties_df.loc[j] = af[j-lower_limit]
    lower_limit += 100

In [26]:
for i in range(len(song_uri_list)//100*100, len(song_uri_list)):
    af = sp.audio_features(song_uri_list[i])
    acoustic_properties_df.loc[i] = af[0]

In [11]:
acoustic_properties_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.719,0.704,1.0,-4.724,1.0,0.0476,0.0691,0.000000,0.1660,0.628,133.002,audio_features,25sgk305KZfyuqVBQIahim,spotify:track:25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436.0,4.0
1,0.724,0.647,1.0,-5.642,1.0,0.0658,0.2800,0.000000,0.1020,0.435,106.960,audio_features,2rPE9A1vEgShuZxxzR2tZH,spotify:track:2rPE9A1vEgShuZxxzR2tZH,https://api.spotify.com/v1/tracks/2rPE9A1vEgSh...,https://api.spotify.com/v1/audio-analysis/2rPE...,207333.0,4.0
2,0.760,0.479,2.0,-5.574,1.0,0.0466,0.5560,0.000000,0.0703,0.913,89.911,audio_features,3KkXRkHbMCARz0aVfEt68P,spotify:track:3KkXRkHbMCARz0aVfEt68P,https://api.spotify.com/v1/tracks/3KkXRkHbMCAR...,https://api.spotify.com/v1/audio-analysis/3KkX...,158040.0,4.0
3,0.752,0.488,6.0,-7.050,1.0,0.0705,0.2970,0.000009,0.0936,0.533,136.041,audio_features,5p7ujcrUXASCNwRaWNHR1C,spotify:track:5p7ujcrUXASCNwRaWNHR1C,https://api.spotify.com/v1/tracks/5p7ujcrUXASC...,https://api.spotify.com/v1/audio-analysis/5p7u...,201661.0,4.0
4,0.833,0.539,11.0,-7.399,0.0,0.1780,0.1630,0.000002,0.1010,0.385,99.947,audio_features,6MWtB6iiXyIwun0YzU6DFP,spotify:track:6MWtB6iiXyIwun0YzU6DFP,https://api.spotify.com/v1/tracks/6MWtB6iiXyIw...,https://api.spotify.com/v1/audio-analysis/6MWt...,149520.0,4.0


In [81]:
# Verifying the consistency of the number of songs

len(acoustic_properties_df['uri'].unique())

6473

In [29]:
acoustic_properties_df.to_csv('D:/Disseratation_data/acoustic_song_properties.csv', index = False)

In [23]:
# Gathering general information about songs through Spotify API

song_info = pd.DataFrame(columns=['uri', 'album_type', 'popularity', 'release_date'])
lower_limit = 0
upper_limit = len(song_uri_list)
k = 0
af=[]
for i in range(50, upper_limit, 50):
    track_info = sp.tracks(song_uri_list[lower_limit:min(i, upper_limit)])
    for j in range(lower_limit, min(i, upper_limit)):
        song_info.loc[j] = {'uri' : track_info['tracks'][j - lower_limit]['uri'], 'album_type': track_info['tracks'][j - lower_limit]['album']['album_type'], 
        'popularity': track_info['tracks'][j - lower_limit]['popularity'], 'release_date' : track_info['tracks'][j - lower_limit]['album']['release_date']}
    lower_limit += 50


In [24]:
for i in range(len(song_uri_list)//50*50, len(song_uri_list)):
    track_info = sp.track(song_uri_list[i])
    song_info.loc[i] = {'uri' : track_info['uri'], 'album_type': track_info['album']['album_type'], 'popularity': track_info['popularity'], 'release_date' : track_info['album']['release_date']}

In [10]:
song_info.head()

,uri,album_type,popularity,release_date
0,spotify:track:25sgk305KZfyuqVBQIahim,single,3,2018-08-17
1,spotify:track:2rPE9A1vEgShuZxxzR2tZH,single,22,2018-11-03
2,spotify:track:3KkXRkHbMCARz0aVfEt68P,compilation,89,2018-12-14
3,spotify:track:5p7ujcrUXASCNwRaWNHR1C,single,77,2018-10-04
4,spotify:track:6MWtB6iiXyIwun0YzU6DFP,single,1,2018-12-24


In [86]:
# Verifying the consistency of the number of songs

len(song_info['uri'].unique())

6473

In [87]:
song_info.to_csv('D:/Disseratation_data/song_info.csv', index = False)

In [27]:
# Merging daily charts with song properties into new data frame combined_df

combined_df = pd.merge(daily_200_combined, acoustic_properties_df, on ='uri', how = 'inner')

In [28]:
len(combined_df)

292200

In [31]:
# Merging song_info with combined_df

combined_df = pd.merge(combined_df, song_info, on ='uri', how = 'inner')

In [32]:
len(combined_df)

292200

In [33]:
# Listing the columns of the combined dataframe

combined_df.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'days_on_chart', 'streams', 'Date', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'album_type', 'popularity', 'release_date'],
      dtype='object')

In [34]:
combined_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,days_on_chart,streams,Date,...,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,album_type,popularity,release_date
0,1,spotify:track:25sgk305KZfyuqVBQIahim,Ava Max,Sweet but Psycho,Atlantic Records,1,1,89,334456,2019-01-01,...,133.002,audio_features,25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436,4,single,3,2018-08-17
1,1,spotify:track:25sgk305KZfyuqVBQIahim,Ava Max,Sweet but Psycho,Atlantic Records,1,1,90,421463,2019-01-02,...,133.002,audio_features,25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436,4,single,3,2018-08-17
2,1,spotify:track:25sgk305KZfyuqVBQIahim,Ava Max,Sweet but Psycho,Atlantic Records,1,1,91,418099,2019-01-03,...,133.002,audio_features,25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436,4,single,3,2018-08-17
3,1,spotify:track:25sgk305KZfyuqVBQIahim,Ava Max,Sweet but Psycho,Atlantic Records,1,1,92,437177,2019-01-04,...,133.002,audio_features,25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436,4,single,3,2018-08-17
4,1,spotify:track:25sgk305KZfyuqVBQIahim,Ava Max,Sweet but Psycho,Atlantic Records,1,1,93,424643,2019-01-05,...,133.002,audio_features,25sgk305KZfyuqVBQIahim,https://api.spotify.com/v1/tracks/25sgk305KZfy...,https://api.spotify.com/v1/audio-analysis/25sg...,187436,4,single,3,2018-08-17


In [35]:
# Saving the combined dataframe

combined_df.to_csv('D:/Disseratation_data/combined_song_data.csv', index = False)

In [94]:
# Reading in Our World In Data Covid-19 data

covid = pd.read_csv("C:/Users/ajaym/Downloads/owid-covid-data.csv")

In [95]:
# Converting date column from string to date datatype

covid['date'] = pd.to_datetime(covid['date'])

In [96]:
# Selecting rows of UK only

uk_covid = covid[covid['iso_code'] == 'GBR']

In [97]:
# Selecting data within the time period of this study

uk_covid = uk_covid[(uk_covid['date'] >= '1/1/2019') & (uk_covid['date'] <= '31/12/2022')]

In [98]:
# Saving the subsetted data

uk_covid.to_csv('D:/Disseratation_data/uk_covid_data.csv', index = False)